In [1]:
import os
import gc
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from timeit import default_timer
start = default_timer()

def start_timer():
    st = default_timer()
    return st
def stop_timer(st):
    runtime = default_timer() - st
    print ("Elapsed time(sec): ", round(runtime,2))

###### PLOTLY #############
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
init_notebook_mode(connected=True)

D:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
st = start_timer()
input_dir = os.path.join(os.pardir, 'D:\Git Workspace\CreditRiskProject\data')
print('Input files:\n{}'.format(os.listdir(input_dir)))
print('Loading data sets...')

sample_size = None
app_train_df = pd.read_csv(os.path.join(input_dir, 'application_train.csv'), nrows=sample_size)
app_test_df = pd.read_csv(os.path.join(input_dir, 'application_test.csv'), nrows=sample_size)
bureau_df = pd.read_csv(os.path.join(input_dir, 'bureau.csv'), nrows=sample_size)
bureau_balance_df = pd.read_csv(os.path.join(input_dir, 'bureau_balance.csv'), nrows=sample_size)
credit_card_df = pd.read_csv(os.path.join(input_dir, 'credit_card_balance.csv'), nrows=sample_size)
pos_cash_df = pd.read_csv(os.path.join(input_dir, 'POS_CASH_balance.csv'), nrows=sample_size)
prev_app_df = pd.read_csv(os.path.join(input_dir, 'previous_application.csv'), nrows=sample_size)
install_df = pd.read_csv(os.path.join(input_dir, 'installments_payments.csv'), nrows=sample_size)
print('Data loaded.\nMain application training data set shape = {}'.format(app_train_df.shape))
print('Main application test data set shape = {}'.format(app_test_df.shape))
print('Positive target proportion = {:.2f}'.format(app_train_df['TARGET'].mean()))
stop_timer(st)

Input files:
['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']
Loading data sets...
Data loaded.
Main application training data set shape = (307511, 122)
Main application test data set shape = (48744, 121)
Positive target proportion = 0.08
Elapsed time(sec):  120.44


In [3]:
app_train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
app_train_df['LOAN_INCOME_RATIO'] = app_train_df['AMT_CREDIT'] / app_train_df['AMT_INCOME_TOTAL']
app_train_df['ANNUITY_INCOME_RATIO'] = app_train_df['AMT_ANNUITY'] / app_train_df['AMT_INCOME_TOTAL']
app_train_df['ANNUITY LENGTH'] = app_train_df['AMT_CREDIT'] / app_train_df['AMT_ANNUITY']

In [5]:
data = [go.Histogram(x=app_train_df[app_train_df["TARGET"]==1]['LOAN_INCOME_RATIO'], name='Target 1'), 
        go.Histogram(x=app_train_df[app_train_df["TARGET"]==0]['LOAN_INCOME_RATIO'], name='Target 0')]

layout = go.Layout(title='Distribution for Loan Income Ratio',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='LOAN_INCOME_RATIO Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\LOAN_INCOME_RATIO Histogram.html'

In [6]:
data = [go.Histogram(x=app_train_df[app_train_df["TARGET"]==1]['ANNUITY_INCOME_RATIO'], name='Target 1'), 
        go.Histogram(x=app_train_df[app_train_df["TARGET"]==0]['ANNUITY_INCOME_RATIO'], name='Target 0')]

layout = go.Layout(title='Distribution for ANNUITY_INCOME_RATIO',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='ANNUITY_INCOME_RATIO Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\ANNUITY_INCOME_RATIO Histogram.html'

In [7]:
data = [go.Histogram(x=app_train_df[app_train_df["TARGET"]==1]['ANNUITY LENGTH'], name='Target 1'), 
        go.Histogram(x=app_train_df[app_train_df["TARGET"]==0]['ANNUITY LENGTH'], name='Target 0')]

layout = go.Layout(title='Distribution for ANNUITY LENGTH',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='ANNUITY LENGTH Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\ANNUITY LENGTH Histogram.html'

In [39]:
# Previous applications
agg_funs = {'SK_ID_CURR': 'count', 'AMT_CREDIT': 'sum'}
prev_apps = prev_app_df.groupby('SK_ID_CURR').agg(agg_funs)
prev_apps.columns = ['PREV APP COUNT', 'TOTAL PREV LOAN AMT']
merged_df = app_train_df.merge(prev_apps, left_on='SK_ID_CURR', right_index=True, how='left')

In [40]:
merged_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,LOAN_INCOME_RATIO,ANNUITY_INCOME_RATIO,ANNUITY LENGTH,PREV APP COUNT,TOTAL PREV LOAN AMT
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,1.0,2.007889,0.121978,16.461104,1.0,179055.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,4.790750,0.132217,36.234085,3.0,1452573.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,2.000000,0.100000,20.000000,1.0,20106.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,2.316167,0.219900,10.532818,9.0,2625259.5
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,4.222222,0.179963,23.461618,6.0,999832.5


In [10]:
data = [go.Histogram(x=merged_df[merged_df["TARGET"]==1]['PREV APP COUNT'], name='Target 1'), 
        go.Histogram(x=merged_df[merged_df["TARGET"]==0]['PREV APP COUNT'], name='Target 0')]

layout = go.Layout(title='Distribution for PREV APP COUNT',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='PREV APP COUNT Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\PREV APP COUNT Histogram.html'

In [11]:
data = [go.Histogram(x=merged_df[merged_df["TARGET"]==1]['TOTAL PREV LOAN AMT'], name='Target 1'), 
        go.Histogram(x=merged_df[merged_df["TARGET"]==0]['TOTAL PREV LOAN AMT'], name='Target 0')]

layout = go.Layout(title='Distribution for TOTAL PREV LOAN AMT',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='TOTAL PREV LOAN AMT Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\TOTAL PREV LOAN AMT Histogram.html'

In [41]:
 # Average the rest of the previous app data
prev_apps_avg = prev_app_df.groupby('SK_ID_CURR').mean()
merged_df = merged_df.merge(prev_apps_avg, left_on='SK_ID_CURR', right_index=True,
                            how='left', suffixes=['', '_PAVG'])

In [42]:
merged_df.columns[-25:]

Index(['RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE_PAVG', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

AMT_ANNUITY_PAVG
AMT_CREDIT_PAVG
AMT_GOODS_PRICE_PAVG
HOUR_APPR_PROCESS_START_PAVG

In [14]:
data = [go.Histogram(x=merged_df[merged_df["TARGET"]==1]['AMT_ANNUITY_PAVG'], name='Target 1'), 
        go.Histogram(x=merged_df[merged_df["TARGET"]==0]['AMT_ANNUITY_PAVG'], name='Target 0')]

layout = go.Layout(title='Distribution for AMT_ANNUITY_PAVG',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='AMT_ANNUITY_PAVG Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\AMT_ANNUITY_PAVG Histogram.html'

In [15]:
data = [go.Histogram(x=merged_df[merged_df["TARGET"]==1]['AMT_CREDIT_PAVG'], name='Target 1'), 
        go.Histogram(x=merged_df[merged_df["TARGET"]==0]['AMT_CREDIT_PAVG'], name='Target 0')]

layout = go.Layout(title='Distribution for AMT_CREDIT_PAVG',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='AMT_CREDIT_PAVG Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\AMT_CREDIT_PAVG Histogram.html'

In [16]:
data = [go.Histogram(x=merged_df[merged_df["TARGET"]==1]['AMT_GOODS_PRICE_PAVG'], name='Target 1'), 
        go.Histogram(x=merged_df[merged_df["TARGET"]==0]['AMT_GOODS_PRICE_PAVG'], name='Target 0')]

layout = go.Layout(title='Distribution for AMT_GOODS_PRICE_PAVG',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='AMT_GOODS_PRICE_PAVG Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\AMT_GOODS_PRICE_PAVG Histogram.html'

In [13]:
data = [go.Histogram(x=merged_df[merged_df["TARGET"]==1]['HOUR_APPR_PROCESS_START_PAVG'], name='Target 1'), 
        go.Histogram(x=merged_df[merged_df["TARGET"]==0]['HOUR_APPR_PROCESS_START_PAVG'], name='Target 0')]

layout = go.Layout(title='Distribution for HOUR_APPR_PROCESS_START_PAVG',barmode='stack')

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='HOUR_APPR_PROCESS_START_PAVG Histogram')

'file://D:\\Git Workspace\\CreditRiskProject\\HOUR_APPR_PROCESS_START_PAVG Histogram.html'

In [43]:
def process_dataframe(input_df, encoder_dict=None):
    """ Process a dataframe into a form useable by LightGBM """

    # Label encode categoricals
    categorical_feats = input_df.columns[input_df.dtypes == 'object']
    categorical_feats = categorical_feats
    encoder_dict = {}
    for feat in categorical_feats:
        encoder = LabelEncoder()
        input_df[feat] = encoder.fit_transform(input_df[feat].fillna('NULL'))
        encoder_dict[feat] = encoder

    return input_df, categorical_feats.tolist(), encoder_dict

In [44]:
# Previous app categorical features
prev_app_df, cat_feats, _ = process_dataframe(prev_app_df)
prev_apps_cat_avg = prev_app_df[cat_feats + ['SK_ID_CURR']].groupby('SK_ID_CURR').agg({k: lambda x: str(x.mode().iloc[0]) for k in cat_feats})
merged_df = merged_df.merge(prev_apps_cat_avg, left_on='SK_ID_CURR', right_index=True,
                        how='left', suffixes=['', '_BAVG'])
print('Shape after merging with previous apps cat data = {}'.format(merged_df.shape))

Shape after merging with previous apps cat data = (307511, 163)


In [45]:
tmp = merged_df.columns[merged_df.dtypes == 'object']
for name in tmp: 
    print(merged_df[name].value_counts())

Cash loans         278232
Revolving loans    29279 
Name: NAME_CONTRACT_TYPE, dtype: int64
F      202448
M      105059
XNA    4     
Name: CODE_GENDER, dtype: int64
N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64
Y    213312
N    94199 
Name: FLAG_OWN_REALTY, dtype: int64
Unaccompanied      248526
Family             40149 
Spouse, partner    11370 
Children           3267  
Other_B            1770  
Other_A            866   
Group of people    271   
Name: NAME_TYPE_SUITE, dtype: int64
Working                 158774
Commercial associate    71617 
Pensioner               55362 
State servant           21703 
Unemployed              22    
Student                 18    
Businessman             10    
Maternity leave         5     
Name: NAME_INCOME_TYPE, dtype: int64
Secondary / secondary special    218391
Higher education                 74863 
Incomplete higher                10277 
Lower secondary                  3816  
Academic degree                  164   
Name: NAME_EDUC

In [17]:
cat_feats

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [46]:
 # Credit card data - numerical features
wm = lambda x: np.average(x, weights=-1/credit_card_df.loc[x.index, 'MONTHS_BALANCE'])
credit_card_avgs = credit_card_df.groupby('SK_ID_CURR').agg(wm)   
merged_df = merged_df.merge(credit_card_avgs, left_on='SK_ID_CURR', right_index=True,
                            how='left', suffixes=['', '_CCAVG'])

In [50]:
tmp = merged_df.columns[merged_df.dtypes == 'object']
names = [name for name in merged_df.columns if '_CCAVF' in name]
credit_card_avgs

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,1489396.0,-2.448980,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
100011,1843384.0,-18.967766,16910.295179,128058.923979,615.170784,615.170784,0.000000,0.000000,NaN,2057.194314,...,16244.769731,16887.870680,16887.870680,0.013670,0.013670,0.000000,0.000000,NaN,0.000000,0.000000
100013,2038692.0,-18.652500,4291.324329,76824.733090,NaN,1368.505798,NaN,NaN,NaN,1924.271728,...,4073.078347,4274.181760,4274.181760,NaN,0.054914,NaN,NaN,NaN,0.002816,0.002816
100021,2594025.0,-6.813332,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
100023,1499902.0,-6.742270,0.000000,160220.577058,NaN,0.000000,NaN,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
100028,1914954.0,-10.939440,15993.090127,225000.000000,NaN,11048.519574,NaN,NaN,NaN,NaN,...,15470.701905,15784.144702,15784.144702,NaN,3.712114,NaN,NaN,NaN,0.000000,0.000000
100036,2621538.0,-5.504249,0.000000,84815.709116,NaN,0.000000,NaN,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
100042,2137382.0,-16.753181,8843.990806,57283.202246,805.742150,1512.040904,614.047839,92.250915,NaN,2087.285884,...,8457.314145,8828.210493,8828.210493,0.058559,0.102645,0.013460,0.030627,NaN,0.004894,0.000000
100043,1557583.0,-8.070831,71027.895576,128945.750800,660.340729,3805.401213,0.000000,3145.060484,NaN,NaN,...,69370.079080,70951.205071,70951.205071,0.019566,0.093678,0.000000,0.074112,NaN,0.000000,0.000000


In [48]:
# Credit card data - categorical features
most_recent_index = credit_card_df.groupby('SK_ID_CURR')['MONTHS_BALANCE'].idxmax()
cat_feats = credit_card_df.columns[credit_card_df.dtypes == 'object'].tolist()  + ['SK_ID_CURR']
merged_df = merged_df.merge(credit_card_df.loc[most_recent_index, cat_feats], left_on='SK_ID_CURR', right_on='SK_ID_CURR',
                   how='left', suffixes=['', '_CCAVG'])
print('Shape after merging with credit card data = {}'.format(merged_df.shape))

Shape after merging with credit card data = (307511, 185)


In [51]:
# Credit bureau data - numerical features
credit_bureau_avgs = bureau_df.groupby('SK_ID_CURR').mean()
merged_df = merged_df.merge(credit_bureau_avgs, left_on='SK_ID_CURR', right_index=True,
                            how='left', suffixes=['', '_BAVG'])
print('Shape after merging with credit bureau data = {}'.format(merged_df.shape))

Shape after merging with credit bureau data = (307511, 198)


In [54]:
# Bureau balance data
most_recent_index = bureau_balance_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].idxmax()
bureau_balance_df = bureau_balance_df.loc[most_recent_index, :]
merged_df = merged_df.merge(bureau_balance_df, left_on='SK_ID_BUREAU', right_on='SK_ID_BUREAU',
                        how='left', suffixes=['', '_B_B'])
print('Shape after merging with bureau balance data = {}'.format(merged_df.shape))

Shape after merging with bureau balance data = (307511, 200)


In [55]:
 # Pos cash data - weight values by recency when averaging
wm = lambda x: np.average(x, weights=-1/pos_cash_df.loc[x.index, 'MONTHS_BALANCE'])
f = {'CNT_INSTALMENT': wm, 'CNT_INSTALMENT_FUTURE': wm, 'SK_DPD': wm, 'SK_DPD_DEF':wm}
cash_avg = pos_cash_df.groupby('SK_ID_CURR')['CNT_INSTALMENT','CNT_INSTALMENT_FUTURE',
                                             'SK_DPD', 'SK_DPD_DEF'].agg(f)
merged_df = merged_df.merge(cash_avg, left_on='SK_ID_CURR', right_index=True,
                            how='left', suffixes=['', '_CAVG'])

In [56]:
# Pos cash data data - categorical features
most_recent_index = pos_cash_df.groupby('SK_ID_CURR')['MONTHS_BALANCE'].idxmax()
cat_feats = pos_cash_df.columns[pos_cash_df.dtypes == 'object'].tolist()  + ['SK_ID_CURR']
merged_df = merged_df.merge(pos_cash_df.loc[most_recent_index, cat_feats], left_on='SK_ID_CURR', right_on='SK_ID_CURR',
                   how='left', suffixes=['', '_CAVG'])
print('Shape after merging with pos cash data = {}'.format(merged_df.shape))

Shape after merging with pos cash data = (307511, 205)


In [57]:
# Installments data
ins_avg = install_df.groupby('SK_ID_CURR').mean()
merged_df = merged_df.merge(ins_avg, left_on='SK_ID_CURR', right_index=True,
                            how='left', suffixes=['', '_IAVG'])
print('Shape after merging with installments data = {}'.format(merged_df.shape))

Shape after merging with installments data = (307511, 212)


In [58]:
# Add more value counts
merged_df = merged_df.merge(pd.DataFrame(bureau_df['SK_ID_CURR'].value_counts()), left_on='SK_ID_CURR', 
                            right_index=True, how='left', suffixes=['', '_CNT_BUREAU'])
merged_df = merged_df.merge(pd.DataFrame(credit_card_df['SK_ID_CURR'].value_counts()), left_on='SK_ID_CURR', 
                            right_index=True, how='left', suffixes=['', '_CNT_CRED_CARD'])
merged_df = merged_df.merge(pd.DataFrame(pos_cash_df['SK_ID_CURR'].value_counts()), left_on='SK_ID_CURR', 
                            right_index=True, how='left', suffixes=['', '_CNT_POS_CASH'])
merged_df = merged_df.merge(pd.DataFrame(install_df['SK_ID_CURR'].value_counts()), left_on='SK_ID_CURR', 
                            right_index=True, how='left', suffixes=['', '_CNT_INSTALL'])
print('Shape after merging with counts data = {}'.format(merged_df.shape))

Shape after merging with counts data = (307511, 216)
